In [ ]:
import pandas as pd
import pyspark
import itertools
from sklearn.utils import shuffle
import numpy as np
frac = 0.001

In [ ]:
test = pd.DataFrame()
postulaciones = pd.read_csv('Sources/fiuba_4_postulaciones_concat.csv')
postulaciones = postulaciones.drop_duplicates(['idaviso','idpostulante'])
postulantes = pd.read_csv('Sources/postulantes_proc.csv')
postulantes = postulantes.drop_duplicates('idpostulante')['idpostulante']
avisos = pd.concat([pd.read_csv('Sources/fiuba_6_avisos_detalle_limpio.csv'),pd.read_csv('Sources/fiuba_5_avisos_concat.csv')])
avisos = avisos.drop_duplicates('idaviso')['idaviso']

In [ ]:
print('postulantes ',postulantes.sample(frac=frac).shape[0])
print('avisos',avisos.sample(frac=frac).shape[0])

In [ ]:
i = 0
print(postulaciones.shape[0],2*postulaciones.shape[0])
while test.shape[0] < 2*postulaciones.shape[0]:
    while test.shape[0] < 2*postulaciones.shape[0]:
        aux = []
        i = i + 1
        postulantes_sample = postulantes.sample(frac=frac)
        avisos_sample = avisos.sample(frac=frac)
        for x in itertools.product(avisos_sample,postulantes_sample):
            aux.append(x)
        aux = pd.DataFrame.from_records(aux)
        aux.rename({0:'idaviso',1:'idpostulante'}, axis=1, inplace=True)
        test = pd.concat([test,aux])
    print('Antes de la poda:', test.shape[0])
    test = test.drop_duplicates(['idaviso','idpostulante'])
    print('Despues de la poda:', test.shape[0])
print(i, test.shape[0],postulaciones.shape[0])

In [ ]:
postulaciones.drop(['Unnamed: 0','fechapostulacion'], axis=1, inplace=True)

In [ ]:
postulaciones['sepostulo'] = 1

In [ ]:
postulaciones = pd.concat([postulaciones,test])

In [ ]:
postulaciones = postulaciones.drop_duplicates(['idaviso','idpostulante'])
postulaciones['sepostulo'].fillna(0,inplace=True)

In [ ]:
test = pd.read_csv('Sources/test_final_100k.csv').drop('id', axis=1)
postualciones = pd.concat([test,postulaciones])
postualciones = postulaciones.drop_duplicates(['idaviso','idpostulante'])
postulaciones = postulaciones[100000:]
postulaciones.shape

In [ ]:
postulaciones = shuffle(postulaciones)

In [ ]:
postulaciones.to_csv('Sources/postulaciones_for_training.csv')